In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain-core langchain-openai pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.7 MB/s eta 0:00:00


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from sklearn.metrics import classification_report
import time
import os
from tqdm import tqdm

In [ ]:
path = '/content/drive/MyDrive/MIAX/TFM/ROBERTA ESG O NO/noticias_totales.csv'
df_trainning = pd.read_csv(path)
df_trainning

<ipython-input-14-dc78bb11701a>:2: DtypeWarning: Columns (5,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_trainning = pd.read_csv(path)


,id,Publication Date,title,content,source,sentiment_body,sentiment_body_score,company,sector,url,ticker,description
0,1.0,2024-12-09 08:41:30+00:00,Call of Duty: Black Ops 6 Review,The last year or so wasn’t the kindest to Call...,Eggplante,positive,0.98,activision,Communication,NaN,ATVI,NaN
1,2.0,2024-07-23 17:13:37+00:00,Xbox Celebrates the Release of Modern Warfare ...,"Recently, many rumors were running on the inte...",Cinelinx,positive,0.90,activision,Communication,NaN,ATVI,NaN
2,3.0,2024-07-18 14:19:34+00:00,How ToxMod's AI impacted toxicity in Call of D...,It's no secret Call of Duty has toxic players....,DNyuz,neutral,0.48,activision,Communication,NaN,ATVI,NaN
3,4.0,2024-07-18 14:04:50+00:00,How ToxMod's AI impacted toxicity in Call of D...,It's no secret Call of Duty has toxic players....,Allusanewshub,neutral,0.54,activision,Communication,NaN,ATVI,NaN
4,5.0,2024-05-27 07:03:20+00:00,"Meta, Activision Sued by Families of Uvalde Sc...",In the wake of the tragic shooting at Robb Ele...,Tech Times,neutral,0.92,activision,Communication,NaN,ATVI,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1070256,NaN,2025-03-21 16:21:09,Nike shares tumble as tariff concerns shake in...,"Shares in NIKE, Inc. (NYSE: NKE) are falling i...",Fast Company,NaN,NaN,nike,NaN,https://www.fastcompany.com/91303743/nike-shar...,NKE,NaN
1070257,NaN,2025-03-21 20:20:23,Making Sense of Early Q1 Earnings Reports,We have been seeing some of the early Q1 resul...,Zacks Investment Research,NaN,NaN,nike,NaN,https://www.zacks.com/commentary/2433762/makin...,NKE,NaN
1070258,NaN,2025-03-22 07:30:00,"Why Is Nike Stock Falling, and Should Investor...",Nike's (NKE -5.37%) sales are declining in eve...,The Motley Fool,NaN,NaN,nike,NaN,https://www.fool.com/investing/2025/03/22/why-...,NKE,NaN
1070259,NaN,2025-03-22 11:00:00,Will Nike Investors' Frustrations End Anytime ...,"If you're a Nike (NKE -5.37%) investor, it wou...",The Motley Fool,NaN,NaN,nike,NaN,https://www.fool.com/investing/2025/03/22/will...,NKE,NaN


In [ ]:
os.environ["OPENAI_API_KEY"] = "xxxxxx"

In [ ]:
news_sample = df_trainning.sample(n=6000, random_state=33).reset_index(drop=True)

In [ ]:
news_sample['title'] = news_sample['title'].fillna('')
news_sample['content'] = news_sample['content'].fillna('')
news_sample['input_text'] = news_sample['title'] + ". " + news_sample['content'].str.slice(0, 300)

In [ ]:
class ESGClassification(BaseModel):
    esg: bool = Field(..., description="True if the news is related to ESG (Environmental, Social, Governance)")

In [ ]:
tagging_prompt = ChatPromptTemplate.from_template(
    """You are an expert in ESG (Environmental, Social, and Governance) analysis.

Analyze the following financial news article and determine whether it is related to ESG topics.

 Please interpret ESG in a broad and inclusive way:
- Environmental includes sustainability, climate change, emissions, energy, biodiversity, etc.
- Social includes diversity, inclusion, employee well-being, human rights, education, community impact, etc.
- Governance includes ethical behavior, transparency, corporate governance, executive compensation, stakeholder rights, etc.

Even if the news is not explicitly labeled as ESG, it may still relate to ESG themes based on its content.

Always respond in English, even if the original article is in another language.

Return:
- `esg`: true or false
Text:
{input}
"""
)

In [ ]:
model = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(ESGClassification)
tagging_chain = tagging_prompt | model

In [ ]:
predictions = []

start = time.time()

for i, row in tqdm(news_sample.iterrows(), total=len(news_sample)):
    text = row['input_text']
    try:
        result = tagging_chain.invoke({"input": text})
        predictions.append(result.esg)
    except Exception as e:
        predictions.append(None)

print("Tiempo total:", round(time.time() - start, 2), "segundos")
news_sample['predicted_esg'] = predictions

100%|██████████| 6000/6000 [1:10:46<00:00,  1.41it/s]

⏱️ Tiempo total: 4246.2 segundos


In [ ]:
news_sample

,id,Publication Date,title,content,source,sentiment_body,sentiment_body_score,company,sector,url,ticker,description,input_text,predicted_esg
0,NaN,2021-04-26T19:00:00Z,Recommended strategies and best practices for ...,Posted by Wally Brill and Jessica Dene Earley-...,Googleblog.com,NaN,NaN,google,NaN,https://developers.googleblog.com/2021/04/Acti...,GOOGL,Here are three key things to keep in mind when...,Recommended strategies and best practices for ...,False
1,333.0,2023-07-10 19:46:51+00:00,The other reason banks aren't making cataclysm...,For all the talk about job cuts in investment ...,EFinancialCareers Italy,negative,0.61,morgan stanley,Finance,NaN,MS,NaN,The other reason banks aren't making cataclysm...,False
2,NaN,2022-06-03T18:00:00Z,"Amazon, Governor Baker Officially Open New Tec...",The latest Amazon office to open in the Seapor...,Yahoo Entertainment,NaN,NaN,amazon,NaN,https://finance.yahoo.com/news/amazon-governor...,AMZN,"BOSTON, June 03, 2022--Today Amazon, along wit...","Amazon, Governor Baker Officially Open New Tec...",True
3,NaN,2021-09-23T10:00:41Z,Using an Amazon Registry So Friends and Family...,"If you buy something through our links, we may...",Small Business Trends,NaN,NaN,amazon,NaN,https://smallbiztrends.com/2021/09/amazon-regi...,AMZN,Using an Amazon registry so friends and family...,Using an Amazon Registry So Friends and Family...,False
4,11800.0,2023-09-08 14:00:53+00:00,Unraveling the European Agricultural Policy Su...,Open Access Article by Yannis E. Doukas Yan...,International Journal of Molecular Sciences,NaN,NaN,google,Technology,https://www.mdpi.com/2073-445X/12/9/1749,GOOGL,NaN,Unraveling the European Agricultural Policy Su...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,NaN,2021-06-01T16:12:07Z,Coinbase Card Now Accepted By Google And Apple...,Coinbase announced integration with Google and...,Yahoo Entertainment,NaN,NaN,apple,NaN,https://finance.yahoo.com/news/coinbase-card-n...,AAPL,Coinbase tests the roll-out of its crypto card...,Coinbase Card Now Accepted By Google And Apple...,False
5996,NaN,2021-10-08T17:43:02Z,Twitch got e-vandalized overnight with photos ...,(Bigstock Photo).\r\nIf you were using the web...,GeekWire,NaN,NaN,amazon,NaN,https://www.geekwire.com/2021/twitch-got-e-van...,AMZN,If you were using the web version of Twitch ea...,Twitch got e-vandalized overnight with photos ...,True
5997,NaN,2020-12-09T21:30:21Z,"Pixel 2 XL is the Best Pixel of All Time, Right?",We said “Goodbye!” to continued software suppo...,Droid Life,NaN,NaN,google,NaN,https://www.droid-life.com/2020/12/09/pixel-2-...,GOOGL,We said “Goodbye!” to continued software suppo...,"Pixel 2 XL is the Best Pixel of All Time, Righ...",False
5998,NaN,2020-06-25T10:42:56Z,Huawei P40 Pro series gets EMUI 10.1.0.140 wit...,Huawei has been striving to give several reaso...,XDA Developers,NaN,NaN,google,NaN,https://www.xda-developers.com/?p=310535,GOOGL,Huawei has been striving to give several reaso...,Huawei P40 Pro series gets EMUI 10.1.0.140 wit...,False


In [ ]:
news_sample['predicted_esg'].value_counts()

,count
predicted_esg,
False,3671
True,2329


In [ ]:
news_sample.to_csv('/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/sample_esg_or_not.csv', index=False)

In [ ]:
print("Ejemplos clasificados como ESG (True):")
display(news_sample[news_sample['predicted_esg'] == True].sample(3, random_state=42)[['title', 'content', 'predicted_esg']])

print("Ejemplos clasificados como No ESG (False):")
display(news_sample[news_sample['predicted_esg'] == False].sample(3, random_state=7)[['title', 'content', 'predicted_esg']])

Ejemplos clasificados como ESG (True):


,title,content,predicted_esg
4211,What to know about rare blood clotting connect...,Federal health officials urged vaccination cen...,True
1619,New Lockdown Measures Close All Remaining Appl...,Apple has temporarily closed all of its stores...,True
4260,Amazon Pushes Further into Space With New AWS ...,An AWS Ground Station. Photo: AWS\r\nAmazon We...,True


Ejemplos clasificados como No ESG (False):


,title,content,predicted_esg
649,"Cloud, not consoles, blocks Microsoft's Activi...",For all the thunder about Xbox versus PlayStat...,False
819,Starfield and Redfall are getting full Japanes...,Two of the biggest upcoming first-party Xbox g...,False
3465,Microsoft’s new Xbox Wireless Headset has been...,Lenovo currently offers the new Microsoft Xbox...,False


In [ ]:
display(news_sample[news_sample['predicted_esg'] == True].sample(3, random_state=55)[['title', 'content', 'predicted_esg']])

,title,content,predicted_esg
1288,Zero Waste Packaging Market Is Booming Worldwi...,Zero Waste Packaging Stay up-to-date with Glo...,True
3429,Online movie groups keep fans engaged while th...,Coronavirus misinformation spread by Russian a...,True
4317,Morning Coffee: Citigroup's job cuts are an ex...,"In its way, Citigroup is lucky. When JPMorgan ...",True
